In [3]:
"""setup"""
from harness import Harness
from harness.dataframe import SeriesBase, FrameBase
import pandas
from sklearn import *
import sklearn
from pandas import np
from typing import Iterable
from whatever.callables import DictCallable
from toolz.curried import *
from whatever import *
import time
from typing import Callable
__all__ = [
    'Harness'
]

ImportError: cannot import name 'dataframe'

In [4]:
"""test_simple_series"""
class MySeries(SeriesBase):
    _metadata = ['col']
class MyFrame(FrameBase):
    _metadata = ['col']
    
MyFrame._series = MySeries
MySeries._dataframe = MyFrame

In [5]:
"""test_harness"""
harness = Harness(
    datasets.load_digits()['data'], index=datasets.load_digits()['target'],
    pipeline=[decomposition.PCA(), preprocessing.Normalizer()]).fit()

In [6]:
"""test_transform_and_score"""
transform = harness.transform()
transform.n_folds=4

transform.pipeline = [
    tree.DecisionTreeClassifier(), tree.ExtraTreeClassifier()
]

transform.fit().score()

,data,length,model,score,time
4559926328,0 1 2 3 4 5 6 7...,1.0,"PCA(copy=True, iterated_power='auto', n_compon...",NaN,0.016619
4822931832,0 1 2 3 4 5 6 7...,1.0,"Normalizer(copy=True, norm='l2')",NaN,0.002616
4822932392,4559926328 ...,1.0,"DecisionTreeClassifier(class_weight=None, crit...",0.957151,0.089288
4822933176,4559926328 ...,1.0,"ExtraTreeClassifier(class_weight=None, criteri...",0.923205,0.003235
4822931664,4559926328 ...,1.0,"DecisionTreeClassifier(class_weight=None, crit...",0.940456,0.084641
4819644712,4559926328 ...,1.0,"ExtraTreeClassifier(class_weight=None, criteri...",0.923205,0.005575
4819644824,4559926328 ...,1.0,"DecisionTreeClassifier(class_weight=None, crit...",0.957707,0.090016
4819644880,4559926328 ...,1.0,"ExtraTreeClassifier(class_weight=None, criteri...",0.943795,0.003293
4822932896,4559926328 ...,1.0,"DecisionTreeClassifier(class_weight=None, crit...",0.948247,0.101872
4822933064,4559926328 ...,1.0,"ExtraTreeClassifier(class_weight=None, criteri...",0.937117,0.003451


In [7]:
"""test_digits_fit"""
data = pandas.DataFrame(datasets.load_digits()['data'], index=datasets.load_digits()['target'])
harness = data.pipe(
    Harness, n_folds=1, pipeline=[
        preprocessing.RobustScaler(),
    ]
)
harness.fit().head(2)

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,0.0,0.0,5.0,13.0,9.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,6.0,13.0,10.0,0.0,0.0,0.0
1,0.0,0.0,0.0,12.0,13.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,11.0,16.0,10.0,0.0,0.0


In [8]:
"""test_digits_scaling"""
scaled = harness.transform()
scaled.head(2)

4819612448                                                     ...   \
          0    1      2    3    4         5    6    7    8    9  ...    
0        0.0  0.0  0.125  0.0 -0.8 -0.272727  0.0  0.0  0.0  0.0 ...    
1        0.0  0.0 -0.500 -0.2  0.0  0.090909  0.0  0.0  0.0  0.0 ...    

                                                                         
         54   55   56   57        58   59        60        61   62   63  
0 -0.142857  0.0  0.0  0.0  0.222222  0.0 -0.666667 -0.500000  0.0  0.0  
1 -0.142857  0.0  0.0  0.0 -0.444444 -0.4  0.333333  0.333333  0.0  0.0  

[2 rows x 64 columns]

In [9]:
"""test_decomposition_on_digits"""
decomposed = scaled.pipe(
    Harness, n_folds=1, pipeline=[
        decomposition.PCA(), feature_selection.VarianceThreshold(.3),
    ]
).fit()
decomposed = decomposed.transform([[decomposed.matrix[0,1]]])

In [10]:
"""test_classifiy_digits"""
classify = decomposed.pipe(
    Harness, n_folds=3, pipeline=[
        tree.ExtraTreeClassifier(), tree.DecisionTreeClassifier(),
    ]
).fit()
classify.score()

,data,length,model,score,time
4822570992,4822570544 ...,1.0,"ExtraTreeClassifier(class_weight=None, criteri...",0.8798,0.003211
4822572896,4822570544 ...,1.0,"DecisionTreeClassifier(class_weight=None, crit...",0.896494,0.008139
4819644992,4822570544 ...,1.0,"ExtraTreeClassifier(class_weight=None, criteri...",0.869783,0.002694
4819648408,4822570544 ...,1.0,"DecisionTreeClassifier(class_weight=None, crit...",0.913745,0.009490
4819648016,4822570544 ...,1.0,"ExtraTreeClassifier(class_weight=None, criteri...",0.883695,0.002893
4819645776,4822570544 ...,1.0,"DecisionTreeClassifier(class_weight=None, crit...",0.91931,0.007402


In [ ]:
"""setup"""
import pandas
from sklearn import *
import sklearn
from pandas import np
from typing import Iterable
from whatever.callables import DictCallable
from toolz.curried import *
from whatever import *
import time
from typing import Callable
__all__ = [
    'Harness'
]